In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
dir1 = "base/varroas"
dir2 = "base/non_varroas"

IMG_WIDTH = 100  # Largeur souhaitée de l'image
IMG_HEIGHT = 100  # Hauteur souhaitée de l'image

X = []
y = []

# Parcourir le premier répertoire
for filename in os.listdir(dir1):
    img_path = os.path.join(dir1, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))  # Redimensionner l'image
    # Effectuez ici d'autres traitements sur l'image si nécessaire
    X.append(img)
    y.append(0)  # 0 pour le premier répertoire

# Parcourir le deuxième répertoire
for filename in os.listdir(dir2):
    img_path = os.path.join(dir2, filename)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))  # Redimensionner l'image
    # Effectuez ici d'autres traitements sur l'image si nécessaire
    X.append(img)
    y.append(1)  # 1 pour le deuxième répertoire

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#image = X_train[2]
# Afficher l'image redimensionnée
#cv2.imshow("Image redimensionnée", image)




In [4]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3) ,input_shape = (100,100,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3) ,activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(3,3) ,activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=20)


Epoch 1/10
32/32 [==============================] - 8s 193ms/step - loss: 4.3430 - accuracy: 0.6183
Epoch 2/10
32/32 [==============================] - 6s 191ms/step - loss: 0.5283 - accuracy: 0.7271
Epoch 3/10
32/32 [==============================] - 6s 191ms/step - loss: 0.4828 - accuracy: 0.7666
Epoch 4/10
32/32 [==============================] - 6s 191ms/step - loss: 0.3709 - accuracy: 0.8549
Epoch 5/10
32/32 [==============================] - 6s 190ms/step - loss: 0.3403 - accuracy: 0.8438
Epoch 6/10
32/32 [==============================] - 6s 194ms/step - loss: 0.3353 - accuracy: 0.8391
Epoch 7/10
32/32 [==============================] - 6s 194ms/step - loss: 0.3020 - accuracy: 0.8770
Epoch 8/10
32/32 [==============================] - 6s 188ms/step - loss: 0.2765 - accuracy: 0.8785
Epoch 9/10
32/32 [==============================] - 6s 195ms/step - loss: 0.2380 - accuracy: 0.9085
Epoch 10/10
32/32 [==============================] - 7s 212ms/step - loss: 0.2802 - accuracy: 0.8707

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

9/9 [==============================] - 1s 62ms/step - loss: 0.2599 - accuracy: 0.8938
Test accuracy: 0.8937729001045227
